In [7]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
from datetime import datetime

In [8]:
result_df = pd.DataFrame()

In [9]:
news=['https://search.naver.com/search.naver?where=news&query=%ED%95%9C%EA%B0%95%20%EC%8B%A4%EC%A2%85%20%EB%8C%80%ED%95%99%EC%83%9D&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds=2021.04.25&de=2021.05.31&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20210425to20210531,a:all','https://search.naver.com/search.naver?where=news&query=%ED%95%9C%EA%B0%95%20%EC%8B%A4%EC%A2%85%20%EB%8C%80%ED%95%99%EC%83%9D&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds=2021.06.01&de=2021.07.31&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20210601to20210731,a:all','https://search.naver.com/search.naver?where=news&query=%ED%95%9C%EA%B0%95%20%EC%8B%A4%EC%A2%85%20%EB%8C%80%ED%95%99%EC%83%9D&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds=2021.08.01&de=2021.09.30&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20210801to20210930,a:all']

In [10]:
start = 1
maxpage=400
maxpage_t =(int(maxpage)-1)*10+1 
# 11= 2페이지 21=3페이지 31=4페이지 ...81=9페이지 , 91=10페이지, 101=11페이지 

while start <= maxpage_t: 
    url=news[2]+"&start="+str(start)
    #print(url)
    headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36','accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')
    news_title = [title['title'] for title in soup.find_all('a', attrs={'class':'news_tit'})] # 기사 제목
    news_url = [ url['href'] for url in soup.find_all('a', attrs={'class':'news_tit'}) ] # 기사 url

    dates = [ date.get_text() for date in soup.find_all('span', attrs={'class':'info'})] # 기사 작성일
    news_date = []
    for date in dates:
        if re.search(r'\d+.\d+.\d+.', date) != None: # 기사 작성일 정제
            news_date.append(date)

    df = pd.DataFrame({'기사작성일':news_date,'기사제목':news_title,'기사주소':news_url})
    result_df = pd.concat([result_df, df], ignore_index=True)
    start += 10

print(start)

4001


In [20]:
now = datetime.now()
outputFileName = '%s-%s-%s %s시 %s분 %s초.xlsx' % (now.year, now.month, now.day, now.hour, now.minute, now.second) 
result_df.to_excel(outputFileName,sheet_name='sheet1')

여기서부터는 xlsx->mysql code

In [32]:
data=pd.read_excel('news_result.xlsx',names = ['I', 'DATE','TITLE','URL'],index_col='I')

In [33]:
import pymysql

conn = pymysql.connect(host='localhost', user='root', password='0000', db='twitter')
mycursor = conn.cursor(pymysql.cursors.DictCursor)

init="DROP TABLES IF EXISTS NEWS"
mycursor.execute(init)

sql="CREATE TABLE NEWS(I INT NOT NULL AUTO_INCREMENT,DATE DATE,TITLE VARCHAR(60),URL VARCHAR(100),PRIMARY KEY(I));"
mycursor.execute(sql)

ins="INSERT INTO NEWS(I,DATE,TITLE,URL) VALUES(NULL,%s,%s,%s)"

for idx in range(len(data)):
    mycursor.execute(ins, tuple(data.values[idx]))
conn.commit()

conn.close()
